# Cythonによる高速化とC/C++とPythonの連携 #

大橋 宏正
twitter, github: @wrist

## 自己紹介 ##

* 収音系の音響信号処理処理が専門のエンジニアです(某メーカー勤務)
* よく使う標準モジュール
    * StringIO
    * wave

## 宣伝 ##

* 書籍「行列プログラマー」のオンライン読書会をやっています
* https://osaka-prml-reading.connpass.com/event/48885/
* 毎週土曜夜21:00-23:00に開催
* トチ狂って本日21時からもやることになっています

## 本日の資料 ##

* https://github.com/wrist/pydata-osaka3

## 今回の発表の背景 ##

* 業務上Pythonでアルゴリズム検討の上、C/C++でプロトタイプ実装を行うことが多い
* 今回の発表の個人的な目的
    * アルゴリズム検討の高速化
        * 信号処理はサンプル/フレーム単位の積和処理ばかりだがPythonだとループが遅い
    * 既存C/C++資産の活用

## Cythonとは ##

* PythonとC/C++レベルでの静的型付けを混在可能な言語
    * コンパイルするとPython/C APIを使用するC/C++コードを生成
    * 生成されたC/C++コードをビルドするとPythonから使用可能な拡張モジュールを生成
* 使用目的
    * 静的型付けによる最適化・高速化
    * C/C++とのグルー言語

## cython.orgのトップページから引用・意訳 ##

* http://cython.org/
*  CythonではPythonとCの融合によって以下のようなことを可能とする
    * C/C++で書かれたネイティブコードを呼ぶPythonコードを書く
    * 簡単に静的な型定義を可読性の高いPythonコードに加えることによってCのパフォーマンスを得る
    * Python、Cython, Cのコード中のバグを見つけるために結合したコードレベルでのデバックを行う???
    * 多次元Numpy配列のような大きなデータセットに対しても効率的なやり取りが可能
    * CPythonに対する広く成熟したエコシステムを活用可能
    * 既存のネイティブコードや、低レベル・ハイパフォ―マンスなライブラリやアプリを統合可能

## どこで使われているか ##

* オライリーのCython本P.71より引用
    * 2014年のデータなので現在は異なる筈
    * https://github.com/cython/cython/wiki/projects にも情報あり

<img src="./img/where_uses_cython.png" />

## インストール ##

* `pip install cython`
* `conda install cython`
    * anacondaなら最初から入っている筈

## Cythonの実際の例 ##

* フィボナッチ数列を求めるPythonコードをCython化

In [1]:
# Pythonで書いたフィボナッチ数列
def fib(n):
    a, b = 0, 1
    for i in range(n):
        a, b = a + b,  a
    return a

In [2]:
fib(50)

12586269025

In [3]:
%timeit fib(50)

100000 loops, best of 3: 4.41 µs per loop


In [4]:
%load_ext cython

In [5]:
%%cython
# .pyxファイルの中身に相当
def c_fib(long n):
    cdef long i, a, b
    a, b = 0, 1
    for i in range(n):
        a, b = a + b, a
    return a

In [6]:
# python側から呼べる
c_fib(50)

12586269025

In [7]:
%timeit c_fib(50)

The slowest run took 22.00 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 114 ns per loop


## 動作イメージ ##

<img src="./img/build_flow.png" />

## 手動でのコンパイルおよびビルド ##

* https://cython.readthedocs.io/en/latest/src/reference/compilation.html
* `cython`コマンドで`.pyx`ファイルをコンパイル
    * `cython -a yourmod.pyx`
* `.pyx`ファイルをダイナミックライブラリとしてビルド
    * `gcc -shared -pthread -fPIC -fwrapv -O2 -Wall -fno-strict-aliasing \
      -I/usr/include/python2.7 -o yourmod.so yourmod.c`

In [8]:
!ls

cython_and_c_wrapper.ipynb pysndfile
docker                     readme.md
img                        rectangle


In [9]:
%%writefile c_fib.pyx
def c_fib(long n):
    cdef long i, a, b
    a, b = 0, 1
    for i in range(n):
        a, b = a + b, a
    return a

Writing c_fib.pyx


In [10]:
!cython c_fib.pyx

In [11]:
!ls

c_fib.c                    img
c_fib.pyx                  pysndfile
cython_and_c_wrapper.ipynb readme.md
docker                     rectangle


生成されたc_fib.cをダイナミックライブラリとしてビルドすることで拡張モジュールを作成可能

## 代表的なビルド方法 ##

* IPythonの`%%cython`マジックコマンド
    * セルの内容をpyxファイルとみなしてその場でコンパイル
* `import pyximport; pyximport.install()`
    * 現在のディレクトリ配下にあるpyxファイルをコンパイル
* ビルドスクリプト`setup.py`を記述(後述)
    * `python setup.py build_ext --inplace`

## 本日の内容 ##

* `%%cython`マジックコマンドによるJupyter notebook上での高速化
    * timeitによる計測とアノテーションによるボトルネック把握を通じた改善
* Cコードに対するラッパー作成方法の紹介
    * Audio IOライブラリ`libsndfile`に対する簡易Cラッパーの作成
* C++コードに対するラッパー作成方法の紹介
    * 公式ドキュメントの内容

# `%%cython`マジックコマンドによるJupyter上での高速化 #

## staicライブラリやdynamicライブラリの使用 ##

* Extensionを作成するときの引数に指定可能
* static library
    * extra_objects
* dynamic library
    * libraries

### ラッパーを手で書くのがめんどくさい ###

* 自動化ツールが存在
    * https://github.com/cython/cython/wiki/AutoPxd
    * メンテされているプロジェクトがあまりない印象

## 情報 ##

* wiki https://github.com/cython/cython/wiki
* document
    * 和訳されたものもあるがバージョンが古い(17.1)ため注意
* オライリーのcython本